In [ ]:
%pip install PyMuPDF llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

separate functions for komatsu doc


combining new info


In [ ]:
import fitz  # PyMuPDF
from google.colab import userdata
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
warnings.filterwarnings('ignore')

# Retrieve Groq API key from Google Colab secrets
GROQ_API_KEY = userdata.get('groq')
if not GROQ_API_KEY:
    raise ValueError("Please set the 'groq_api_key' secret in Google Colab.")

# Function to extract text from a PDF file using PyMuPDF
def extract_text_from_pdf(pdf_file):
    text = ''
    doc = fitz.open(pdf_file)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    doc.close()
    return text

# Function to merge new information into the text
def merge_new_info(snippets, new_info):
    """Merge new information into the matching snippets to create a coherent text."""
    if not snippets:
        return new_info
    combined_text = " ".join([snippet.strip() for snippet in snippets if snippet.strip()])
    combined_text += f" {new_info}"
    return combined_text

# Extract text from the PDF file
pdf_file_name = 'komatsu_doc.pdf'
text_file = extract_text_from_pdf(pdf_file_name)

# Save extracted text to a temporary text file for ingestion
with open("komatsu_doc.txt", "w") as file:
    file.write(text_file)

# Define paths for the text file
input_file = "komatsu_doc.txt"

# Load document
reader = SimpleDirectoryReader(input_files=[input_file])
documents = reader.load_data()

# Initialize SentenceSplitter and get nodes from documents
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

# Initialize HuggingFace embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize Groq LLM model
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

# Configure ServiceContext
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

# Create Vector Store Index
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

# Persist and save index
vector_index.storage_context.persist(persist_dir="./storage_mini")

# Define storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

# Load index
index = load_index_from_storage(storage_context, service_context=service_context)

# Initialize query engine
query_engine = index.as_query_engine(service_context=service_context)

# Define the new information to match against the document
new_info = 'which reduces human efficiency and synchronization period.'

# Query to find matching snippets for the new information
query = "Find matching snippets related to the IntelliMine Adapter forces resynchronization every 24 hours"

# Query the index to find matching snippets for the new information
response = query_engine.query(query)

# Extract snippets from response
matching_snippets = response.response.split('\n')

# Merge new information with matching snippets to create coherent text
combined_text = merge_new_info(matching_snippets, new_info)

# Use the query engine to rewrite the combined text
def rewrite_with_query_engine(text, query_engine):
    prompt = f"Rewrite the following text into a coherent paragraph without adding any emotional or unnecessary content:\n\n{text}"
    response = query_engine.query(prompt)
    return response.response.strip()

# Rewrite the combined text using the query engine
updated_text = rewrite_with_query_engine(combined_text, query_engine)

# Output the updated text
print("Updated Text for Documentation File:")
print(updated_text)



Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Updated Text for Documentation File:
IntelliMine Adapter is set to force resynchronization every 24 hours by default, serving as a fail-safe measure in case it fails to detect an equipment configuration change. This default setting can be modified to increase the resynchronization frequency, which can help improve efficiency and reduce the synchronization period.


replacing text with link

In [ ]:
import fitz  # PyMuPDF
from google.colab import userdata
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
warnings.filterwarnings('ignore')

# Retrieve Groq API key from Google Colab secrets
GROQ_API_KEY = userdata.get('groq')
if not GROQ_API_KEY:
    raise ValueError("Please set the 'groq_api_key' secret in Google Colab.")

# Function to extract text from a PDF file using PyMuPDF
def extract_text_from_pdf(pdf_file):
    text = ''
    doc = fitz.open(pdf_file)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    doc.close()
    return text

# Function to extract snippets containing specific keywords
def extract_snippets(text, keywords):
    """Extract snippets from the text containing any of the specified keywords."""
    snippets = []
    lines = text.split('\n')
    for line in lines:
        if any(keyword in line for keyword in keywords):
            snippets.append(line)
    return snippets

# Function to change text to link
def change_text_to_link(text, variants, link):
    """Replace specified text variants with a hyperlink."""
    for variant in variants:
        text = text.replace(variant, link)
    return text

# Extract text from the PDF file
pdf_file_name = 'komatsu_doc.pdf'
text_file = extract_text_from_pdf(pdf_file_name)

# Save extracted text to a temporary text file for ingestion
with open("komatsu_doc.txt", "w") as file:
    file.write(text_file)

# Define paths for the text file
input_file = "komatsu_doc.txt"

# Load document
reader = SimpleDirectoryReader(input_files=[input_file])
documents = reader.load_data()

# Initialize SentenceSplitter and get nodes from documents
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

# Initialize HuggingFace embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize Groq LLM model
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

# Configure ServiceContext
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

# Create Vector Store Index
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

# Persist and save index
vector_index.storage_context.persist(persist_dir="./storage_mini")

# Define storage context
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")

# Load index
index = load_index_from_storage(storage_context, service_context=service_context)

# Initialize query engine
query_engine = index.as_query_engine(service_context=service_context)

# Extract snippets containing specific keywords
keywords = ['Modular']
extracted_snippets = extract_snippets(text_file, keywords)

# Convert extracted snippets to text
text_with_snippets = '\n'.join(extracted_snippets)

# Define the link to replace the keyword
mining_link = '<a href="https://en.wikipedia.org/wiki/leaders"></a>'

# Change text to link in the extracted snippets
text_with_links = change_text_to_link(text_with_snippets, keywords, mining_link)

# Save and reload updated text
with open("updated_komatsu_doc.txt", "w") as file:
    file.write(text_with_links)

print("Text with links:")
print(text_with_links)


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Text with links:
The Natural Data Selectors option allows <a href="https://en.wikipedia.org/wiki/leaders"></a> personnel to capture real-time
troubleshooting. <a href="https://en.wikipedia.org/wiki/leaders"></a> personnel create a natural data selector to specify the
●If <a href="https://en.wikipedia.org/wiki/leaders"></a> Mining Analytics Solutions is integrated with the MineCare
The Aggregated Data Selectors option allows <a href="https://en.wikipedia.org/wiki/leaders"></a> personnel to collect and


remove terms

In [ ]:
import fitz  # PyMuPDF
from google.colab import userdata
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
warnings.filterwarnings('ignore')

# Retrieve Groq API key from Google Colab secrets
GROQ_API_KEY = userdata.get('groq')
if not GROQ_API_KEY:
    raise ValueError("Please set the 'groq_api_key' secret in Google Colab.")

# Function to extract text from a PDF file using PyMuPDF
def extract_text_from_pdf(pdf_file):
    text = ''
    doc = fitz.open(pdf_file)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    doc.close()
    return text

# Function to extract specific snippets containing the term
def extract_snippets_with_term(text, term):
    """Extract snippets from the text containing the specified term."""
    snippets = []
    lines = text.split('\n')
    for line in lines:
        if term in line:
            snippets.append(line)
    return snippets

# Function to remove specific information, including extra spaces
def remove_information(text, terms_to_remove):
    """Remove specified terms from the text and clean up extra spaces."""
    for term in terms_to_remove:
        # Replace the term with an empty string and remove extra spaces
        text = text.replace(term, '').replace('  ', ' ')
    return text

# Extract text from the PDF file
pdf_file_name = 'komatsu_doc.pdf'
text_file = extract_text_from_pdf(pdf_file_name)

# Extract snippets containing the term 'inclinometer'
term_to_extract = 'inclinometer'
snippets_with_term = extract_snippets_with_term(text_file, term_to_extract)

# Remove the term 'inclinometer' from the text
terms_to_remove = ['inclinometer']
cleaned_snippets = [remove_information(snippet, terms_to_remove) for snippet in snippets_with_term]

# Print or save cleaned snippets
print("Cleaned Snippets:")
for snippet in cleaned_snippets:
    print(snippet)

# Optionally, save these cleaned snippets to a file
with open("cleaned_snippets.txt", "w") as file:
    for snippet in cleaned_snippets:
        file.write(snippet + "\n")


Cleaned Snippets:
The 2-axis Drill Platform (108898) mounts horizontally on the drill
will install the 1-axis Drill Mast (108899) and connect it to the 2-axis
Drill Platform . The 1-axis Drill Mast measures the
inclination of the drill mast. You mount the Drill Platform onto a
the on either set of studs to allow the to meet the
●The (1) must be horizontally oriented and parallel to the
●The input connector (2) must face the open end of the
●The output connector (3) must face the front of the vehicle.


replacing numbers

In [ ]:
import fitz  # PyMuPDF
from google.colab import userdata
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
warnings.filterwarnings('ignore')

# Retrieve Groq API key from Google Colab secrets
GROQ_API_KEY = userdata.get('groq')
if not GROQ_API_KEY:
    raise ValueError("Please set the 'groq_api_key' secret in Google Colab.")

# Function to extract text from a PDF file using PyMuPDF
def extract_text_from_pdf(pdf_file):
    text = ''
    doc = fitz.open(pdf_file)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    doc.close()
    return text

# Function to extract a specific section containing the search term, with full sentences
def extract_full_sentence_with_term(text, search_term):
    """Extract the full sentence or section containing the specific search term."""
    # Split the text into sentences based on periods (.)
    sentences = text.split('.')
    for sentence in sentences:
        if search_term in sentence:
            # Return the full sentence containing the search term
            return sentence.strip() + '.'
    return None

# Function to replace numbers in the text
def replace_number(text, patterns):
    """Replace numbers in the text based on specified patterns."""
    for old_number, new_number in patterns:
        text = text.replace(old_number, new_number)
    return text

# Extract text from the PDF file
pdf_file_name = 'komatsu_doc.pdf'
text_file = extract_text_from_pdf(pdf_file_name)

# Extract the specific section containing the term '410'
search_term = '410'
full_sentence_with_term = extract_full_sentence_with_term(text_file, search_term)

# Define patterns for number replacements
patterns = [('410', '2050')]

# Replace numbers in the extracted section
if full_sentence_with_term:
    updated_text = replace_number(full_sentence_with_term, patterns)
else:
    updated_text = "No matching text found."

# Save the updated text to a temporary text file
with open("updated_snippets.txt", "w") as file:
    file.write(updated_text + "\n")

# Output the updated text
print("Updated Text:")
print(updated_text)


Updated Text:
Note: The max backlight intensity of the PTX10 is much higher than the PTXC10
at around 1200 Nits as compared to 2050 Nits.


updating text testing grammar and verb adjustment

In [ ]:
import fitz  # PyMuPDF
from google.colab import userdata
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
warnings.filterwarnings('ignore')

# Retrieve Groq API key from Google Colab secrets
GROQ_API_KEY = userdata.get('groq')
if not GROQ_API_KEY:
    raise ValueError("Please set the 'groq_api_key' secret in Google Colab.")

# Function to extract text from a PDF file using PyMuPDF
def extract_text_from_pdf(pdf_file):
    text = ''
    doc = fitz.open(pdf_file)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    doc.close()
    return text

# Function to find and extract specific snippet
def extract_specific_snippet(text, specific_text):
    """Extract the snippet from the text that contains the specific text."""
    start_index = text.find(specific_text)
    if start_index == -1:
        return None
    # Extract snippet around the specific text
    snippet_length = 200  # Adjust as needed
    end_index = min(start_index + len(specific_text) + snippet_length, len(text))
    return text[start_index:end_index]

# Function to update text with new text and adjust for grammar and verb agreement
def update_text_with_grammar(text, old_text, new_text):
    """Update specific text with new text and adjust for grammar and verb agreement."""
    if old_text in text:
        updated_text = text.replace(old_text, f"{new_text} and")
    else:
        updated_text = text
    return updated_text

# Extract text from the PDF file
pdf_file_name = 'komatsu_doc.pdf'
text_file = extract_text_from_pdf(pdf_file_name)

# Define the specific text snippet to extract
specific_text = 'PTX10 requires slightly more power input at 16.0W typical'

# Extract the specific snippet
specific_snippet = extract_specific_snippet(text_file, specific_text)

# Define old and new text for updating
old_text = 'slightly more power input at'
new_text = '20W'

# Update the extracted snippet with new text and ensure proper grammar
if specific_snippet:
    updated_snippet = update_text_with_grammar(specific_snippet, old_text, new_text)
    print("Updated Snippet:")
    print(updated_snippet)
else:
    print("Specific snippet not found in the text.")


Updated Snippet:
PTX10 requires 20W and 16.0W typical (as compared to the
PTXC10, which was 12.5W typical). This increases by 10W to 26W when set to
max display brightness. This does not include any additional power consumption
resulting from the use of
